In [1]:
#!pip install bs4
#!pip install requests
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.shape

(103, 3)

In [ ]:
!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [ ]:
#import pandas as pd

#df = pd.read_csv (r'C:\Users\hp\Downloads\Geospatial_Coordinates.csv')
#print (df)

In [ ]:

geo_data = pd.read_csv(r"C:\Users\hp\Downloads\Geospatial_Coordinates.csv")
#print(geo_data)
geo_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
print(geo_data)
toronto_borough = df.merge(geo_data, on=["PostalCode"])
toronto_borough = toronto_borough[toronto_borough['Borough'].str.contains('Toronto', na=False)].reset_index(drop=True)

In [ ]:
toronto_borough.head(12)

In [ ]:
toronto_borough.head(12)

In [ ]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [ ]:
toronto_borough.groupby('Borough').count()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_borough[['Neighborhood','PostalCode']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_borough['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [ ]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()

In [ ]:
# import k-means from clustering stage
#!pip install sklearn
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
label_col = toronto_grouped[["Borough","Cluster Labels"]].set_index('Borough')
# using apply function to create a new column
toronto_borough['labels'] = toronto_borough.apply(lambda row: label_col.loc[row["Borough"], "Cluster Labels"], axis = 1)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_borough['Latitude'], toronto_borough['Longitude'], toronto_borough['Neighborhood'], toronto_borough['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Conclusion:
inference: It's easy to see that the labels are clustering the density of neighborhoods and postal code density for a set of boroughs